# REDCap COVID19-Related Surveys

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Exploration
This notebook is aimed at taking an initial look at the responses from the COVID19-related surveys.

The surveys that seem to contain the most relevant questions are:
1. Environment and Experiences - Event 1
2. Spring Break Questionnaire - Event 1
3. CRISIS Survey - Event 3
4. COVID Survey - Event 3

The questions that are somewhat (missing entries) identified [here](https://docs.google.com/spreadsheets/d/1P2D1W4jXsCPe3w85VLKFKW3rQpuslaYXLfElHlNj9pM/edit#gid=0) (will need access from Hagen).

In [4]:
import os
import sys
sys.path.append('../')

from src.features import build_features
from src.visualization import visualize
from src.reports import make_report

from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates

## Data Import
We do not have processed streams for these data since they are contained in one CSV file. 

In [2]:
import pandas as pd
import numpy as np

### Spring Break Questionnaire
One of the more interesting questionnaires asks participants' changes in choices before, during, and "after" Spring Break 2020. 

We have two csv files, one that corresponds to the labelled question with labelled answers:

In [5]:
sb_labels = pd.read_csv('../data/raw/bpeace2/surveys/SpringBreak_E1_labels.csv')
sb_labels.head()

,Record ID,Event Name,Survey Timestamp,Did you travel out of Austin during the last week of classes (March 9-13th),Reasons for travel (choice=Vacation),Reasons for travel (choice=Return home),Reasons for travel (choice=Work),Reasons for travel (choice=Other),How did you travel (check all that apply)? (choice=Car),How did you travel (check all that apply)? (choice=Bus),...,How (check all that apply)? (choice=Social distancing),How (check all that apply)? (choice=Other),Did you change your activities during the second week of spring break in response to the COVID-19 Outbreak?,How (check all that apply)? (choice=Cancelled vacation).1,How (check all that apply)? (choice=Shortened vacation).1,How (check all that apply)? (choice=Limited travel).1,How (check all that apply)? (choice=Changed activities).1,How (check all that apply)? (choice=Social distancing).1,How (check all that apply)? (choice=Other).1,Complete?
0,1,Event 1,2019-11-10 17:00:23,No,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,...,Checked,Unchecked,Yes,Unchecked,Unchecked,Unchecked,Unchecked,Checked,Unchecked,Complete
1,2,Event 1,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,...,Unchecked,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Incomplete
2,3,Event 1,NaN,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,...,Unchecked,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Incomplete
3,4,Event 1,2019-07-11 15:39:42,No,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,...,Checked,Unchecked,No,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Complete
4,5,Event 1,2020-03-05 18:28:29,No,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,...,Checked,Unchecked,Yes,Checked,Checked,Checked,Checked,Checked,Unchecked,Complete


And another corresponding to simplified questions and coded responses

In [6]:
sb = pd.read_csv('../data/raw/bpeace2/surveys/SpringBreak_E1_raw.csv')
sb.head()

,record_id,redcap_event_name,spring_break_questions_only_once_timestamp,did_you_travel_out_of_aust,reasons_for_travel___1,reasons_for_travel___2,reasons_for_travel___3,reasons_for_travel___4,how_did_you_travel_check_a___1,how_did_you_travel_check_a___2,...,how_check_all_that_apply___5,how_check_all_that_apply___6,change_act_second_week,how_check_all_that_apply2___1,how_check_all_that_apply2___2,how_check_all_that_apply2___3,how_check_all_that_apply2___4,how_check_all_that_apply2___5,how_check_all_that_apply2___6,spring_break_questions_only_once_complete
0,1,event_1_arm_1,2019-11-10 17:00:23,0.0,0,0,0,0,0,0,...,1,0,1.0,0,0,0,0,1,0,2
1,2,event_1_arm_1,NaN,NaN,0,0,0,0,0,0,...,0,0,NaN,0,0,0,0,0,0,0
2,3,event_1_arm_1,NaN,NaN,0,0,0,0,0,0,...,0,0,NaN,0,0,0,0,0,0,0
3,4,event_1_arm_1,2019-07-11 15:39:42,0.0,0,0,0,0,0,0,...,1,0,0.0,0,0,0,0,0,0,2
4,5,event_1_arm_1,2020-03-05 18:28:29,0.0,0,0,0,0,0,0,...,1,0,1.0,1,1,1,1,1,0,2


## Analysis

### Spring Break Questionnaire
Looking at some of the responses to the Spring Break Questionnaire

#### Identifying important columns

Break points for the three separate weeks:
1. **Prior to Spring Break**: Columns 3 - 43
2. **First Week**: Columns 44 - 84
3. **Second Week**: Columns 85 - 120

In [18]:
sb_labels.columns[3:44]

Index(['Did you travel out of Austin during the last week of classes (March 9-13th)',
       'Reasons for travel (choice=Vacation)',
       'Reasons for travel (choice=Return home)',
       'Reasons for travel (choice=Work)', 'Reasons for travel (choice=Other)',
       'How did you travel (check all that apply)? (choice=Car)',
       'How did you travel (check all that apply)? (choice=Bus)',
       'How did you travel (check all that apply)? (choice=Plane)',
       'How did you travel (check all that apply)? (choice=Rideshare)',
       'How did you travel (check all that apply)? (choice=Other)',
       'During the last week of classes, what activities did you participate in: (choice=Cooked dinner at home?)',
       'During the last week of classes, what activities did you participate in: (choice=Picked up take-out food)',
       'During the last week of classes, what activities did you participate in: (choice=Had take-out food delivered to your home?)',
       'During the last week of 

In [ ]:
sb_labels[]

#### Leisure Columns
The following columns identify the leisure categories:
1. Gather in groups
2. Videochat
3. Hosted visitors
4. Park
5. Board GAmes
6. Bar
7. Movie